# Sentiment based music recommendation system

This system is designed to suggest music that fits how you feel and what you are feeling of experiencing in the moment. It pays attention to multiple factors like tone and emotion in what you describe, interprets the situation you are in, and looks for songs that match both the theme and the overall musical mood. After processing the users description, it offers a short list of one to five songs that align with the atmosphere they are trying to create.

To get started with the project, we run the cells in order, describe our situation in our own words, and receive a set of recommendations tailored to our current state of mind or mood.


## 1 - Setup and Imports


In [ ]:
import pandas as pd
import numpy as np
import re
import os
import logging
from pathlib import Path
from typing import Optional, Tuple, Dict, List
import pickle

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer

try:
    from langdetect import detect, DetectorFactory
except ImportError:
    !pip install langdetect
    from langdetect import detect, DetectorFactory

from tqdm import tqdm
import torch

import warnings
warnings.filterwarnings('ignore')

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

SAMPLE_SIZE = 5530
DEFAULT_TOP_N = 5
TEXT_WEIGHT = 0.65
AUDIO_WEIGHT = 0.35
SENTIMENT_WEIGHT = 0.1

SENTIMENT_MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"  # This is the sentiment model were using
EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"

DetectorFactory.seed = 0

print("All libraries have successfully been imported")
print(f"The sentiment model we're using: {SENTIMENT_MODEL}")
print(f"The embedding model were using: {EMBEDDING_MODEL}")

All libraries have successfully been imported
The sentiment model we're using: cardiffnlp/twitter-roberta-base-sentiment-latest
The embedding model were using: sentence-transformers/all-mpnet-base-v2


##2 - Data Loading


In [ ]:
def load_dataset():
    #We're using a Kaggle dataset for our project
    #that has lysics as well for the language processing aspect
    dataset_files = [
        '/content/songs_with_attributes_and_lyrics.csv'
    ]

    for filename in dataset_files:
        if os.path.exists(filename):
            logger.info(f"Loading dataset from: {filename}")
            df = pd.read_csv(filename, on_bad_lines='skip', engine='python')
            print(f"Dataset's been loaded: {df.shape[0]:,} songs, {df.shape[1]} features")
            return df

    raise FileNotFoundError(
        "Cant find 'songs_with_attributes_and_lyrics.csv' in the mentioned directory."
    )


df = load_dataset()
print(f"\nDataset preview:")
df.head(2)

Dataset's been loaded: 955,320 songs, 17 features

Dataset preview:


,id,name,album_name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,lyrics
0,0Prct5TDjAnEgIqbxcldY9,!,UNDEN!ABLE,['HELLYEAH'],0.415,0.605,7,-11.157,1,0.0575,0.00116,0.838,0.471,0.193,100.059,79500.0,"He said he came from Jamaica,\n he owned a cou..."
1,2ASl4wirkeYm3OWZxXKYuq,!!,NaN,Yxngxr1,0.788,0.648,7,-9.135,0,0.3150,0.90000,0.000,0.176,0.287,79.998,114000.0,"Fucked a bitch, now she running with my kids\n..."


## 3 - Data Preprocessing

##3.1 Step 1: Lyrics Cleaning


In [ ]:
def clean_lyrics(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def remove_stopwords(text):
    #We'll remove unnecessary stopwords
    if not text:
        return ""
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

print("Cleaning functions have been successfuly defined")


Cleaning functions have been successfuly defined


## 3.2 Step 2: Apply Lyrics Cleaning


In [ ]:
# Sample the dataset again if needed
if SAMPLE_SIZE and len(df) > SAMPLE_SIZE:
    print(f"Sampling {SAMPLE_SIZE} songs from {len(df):,} total")
    df = df.sample(SAMPLE_SIZE, random_state=42).copy()
    print(f"✓ Sampled dataset: {len(df):,} songs")

print("\nCleaning lyrics:")
df_clean = df[df['lyrics'].notna()].copy()
df_clean['clean_lyrics'] = df_clean['lyrics'].apply(clean_lyrics)
df_clean['clean_lyrics'] = df_clean['clean_lyrics'].apply(remove_stopwords)
df_clean['word_count'] = df_clean['clean_lyrics'].apply(lambda x: len(x.split()))

print(f"Lyrics have been cleaned: {len(df_clean):,} songs with valid lyrics")
print(f"Average word count of valid songs: {df_clean['word_count'].mean():.1f} words")


Sampling 5530 songs from 955,320 total
✓ Sampled dataset: 5,530 songs

Cleaning lyrics:
Lyrics have been cleaned: 5,530 songs with valid lyrics
Average word count of valid songs: 153.9 words


## 3.3 Step 3: Language Detection


In [ ]:
def detect_language_safe(text):
    try:
        return detect(text[:800])
    except:
        return "unknown"

print("Detecting languages for all the songs:")
""" THis might take long based on the processing power
and the size of the dataset"""

df_clean['language'] = df_clean['clean_lyrics'].apply(detect_language_safe)

# Map language codes to names
language_map = {
    'en': 'English', 'es': 'Spansh', 'pt': 'Portuguese',
    'fr': 'French', 'de': 'German', 'it': 'Italian',
    'nl': 'Dutch', 'ru': 'Rusian', 'ja': 'Japanese',
    'ko': 'Korean', 'zh': 'Chinese', 'unknown': 'Unknown'
}
df_clean['language_name'] = df_clean['language'].map(language_map).fillna('Other')

print(f"Language detection done")
print(f"\nLanguage distribution:")
print(df_clean['language_name'].value_counts().head(10))


Detecting languages for all the songs:
Language detection done

Language distribution:
language_name
English       4146
Spansh         434
Other          287
Portuguese     218
Unknown        186
French         129
German          91
Italian         26
Dutch           13
Name: count, dtype: int64


## 4 - Sentiment Analysis

We have done this via **cardiffnlp/twitter-roberta-base-sentiment-latest** - A RoBERTa model traubed and tuned on Twitter data for sentiment analysis.


In [ ]:
#We first load the model and chose the RoBERTa model for its accuracy
print(f"model: {SENTIMENT_MODEL}")


sentiment_model = pipeline(
    "sentiment-analysis",
    model=SENTIMENT_MODEL,
    tokenizer=SENTIMENT_MODEL,
    return_all_scores=False
)

print("Sentiment model has been loaded\n")

# Filter to language = eng songs for sentiment analysis
df_en = df_clean[df_clean['language'] == 'en'].copy() if 'language' in df_clean.columns else df_clean.copy()
print(f"Analyzing sentiment for {len(df_en):,} English songs:")

sentiments = []
for lyric in tqdm(df_en['clean_lyrics'], desc="Sentiment analysis"):
    try:
        result = sentiment_model(lyric[:512])[0]
        # Map labels to standard format
        label = result['label'].upper()
        if 'POSITIVE' in label or 'POS' in label:
            label = 'POSITIVE'
        elif 'NEGATIVE' in label or 'NEG' in label:
            label = 'NEGATIVE'
        else:
            label = 'NEUTRAL'
        sentiments.append({'label': label, 'score': result['score']})
    except Exception as e:
        sentiments.append({'label': 'NEUTRAL', 'score': 0.5})

df_en['sentiment_label'] = [s['label'] for s in sentiments]
df_en['sentiment_score'] = [s['score'] for s in sentiments]

print(f"\Sentiment analysis done")
print(f"\nSentiment distribution:")
print(df_en['sentiment_label'].value_counts(normalize=True).mul(100).round(2))


model: cardiffnlp/twitter-roberta-base-sentiment-latest


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


Sentiment model has been loaded

Analyzing sentiment for 4,146 English songs:


Sentiment analysis: 100%|██████████| 4146/4146 [00:44<00:00, 92.22it/s]

\Sentiment analysis done

Sentiment distribution:
sentiment_label
NEUTRAL     44.60
NEGATIVE    36.40
POSITIVE    19.01
Name: proportion, dtype: float64


##5 - Create Semantic Embeddings

We use the **sentence-transformers/all-mpnet-base-v2** model - A tried and tested model that provides 768 dimensional embeddings for  semantic understanding.


In [ ]:
print("Loading the embedding model:")
print(f"model: {EMBEDDING_MODEL}")
"""As mentioned in the title, the model we chose provides
768 dimensional embeddings for better semantic matching"""
embedding_model = SentenceTransformer(EMBEDDING_MODEL)

print("model loaded\n")
print(f"Creating embeddings for {len(df_en):,} songs:")


embeddings = embedding_model.encode(
    df_en['clean_lyrics'].tolist(),
    show_progress_bar=True,
    batch_size=32,
    convert_to_numpy=True
)

embeddings = np.array(embeddings)
df_en['embedding'] = list(embeddings)

print(f"DOne")
print(f"  Embedding shape: {embeddings.shape}")
print(f"  Embedding dimension: {embeddings.shape[1]} (higher values means better semantic understanding)")


Loading the embedding model:
model: sentence-transformers/all-mpnet-base-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model loaded

Creating embeddings for 4,146 songs:


Batches:   0%|          | 0/130 [00:00<?, ?it/s]

DOne
  Embedding shape: (4146, 768)
  Embedding dimension: 768 (higher values means better semantic understanding)



##6 - Normalise Audio Features


In [ ]:
print("Normalising audio features:")

audio_features = ['valence', 'energy', 'danceability', 'acousticness',
                 'instrumentalness', 'liveness', 'speechiness']
available_features = [f for f in audio_features if f in df_en.columns]

scaler = MinMaxScaler()
audio_scaled = scaler.fit_transform(df_en[available_features])

for i, feature in enumerate(available_features):
    df_en[f'{feature}_scaled'] = audio_scaled[:, i]

print(f"normalised {len(available_features)} audio features")
print(f"\nNormalized feature statistics:")
print(df_en[[f'{f}_scaled' for f in available_features]].describe().round(3))

# Final processed dataset
df_processed = df_en.copy()
print(f"\nProcessing done! Final dataset: {len(df_processed):,} songs are ready for recommendations!")


Normalising audio features:
normalised 7 audio features

Normalized feature statistics:
       valence_scaled  energy_scaled  danceability_scaled  \
count        4146.000       4146.000             4146.000   
mean            0.468          0.651                0.557   
std             0.249          0.243                0.178   
min             0.000          0.000                0.000   
25%             0.265          0.473                0.437   
50%             0.448          0.684                0.561   
75%             0.666          0.864                0.683   
max             1.000          1.000                1.000   

       acousticness_scaled  instrumentalness_scaled  liveness_scaled  \
count             4146.000                 4146.000         4146.000   
mean                 0.267                    0.099            0.201   
std                  0.313                    0.232            0.195   
min                  0.000                    0.000            0.000   
25

##7 - Models Used Summary

###Models implemented as aforementioned:

1. **Sentiment Analysis**: `cardiffnlp/twitter-roberta-base-sentiment-latest` : This version uses an advanced RoBERTa model that has been trained on Twitter data and delivers stronger overall accuracy than DistilBERT.


2. **Semantic Embeddings**: `sentence-transformers/all-mpnet-base-v2` : It produces richer 768 dimensional embeddings that offer stronger semantic understanding and has ranked among the top performers on the MTEB benchmark.
  

3. **Language Detection**: `langdetect` : It provides quick and precise language identification and is capable of recognizing more than fifty five languages.



##8 - How sentiment & context analysis are combined for recommendations

### The Merging Process Explained

The recommendation system combines **three similarity scores** to find the best matches: The recommendation process combines three elements: most of the weight comes from how closely the text or context matches in meaning, followed by a smaller contribution from how similar the musical qualities are, and a final adjustment based on how well the emotional tone aligns. All of this is brought together within the recommend function shown below.


In [ ]:
#HOW SENTIMENT & CONTEXT ANALYSIS MERGE


print("""
MERGING PROCESS:

1. USER INPUT ANALYSIS - It first identifies whether the tone is positive,
negative, or neutral, and then interprets the overall context by creating a
detailed numerical representation of the meaning.


2. SIMILARITY CALCULATIONS - The system places most of its emphasis on how
closely the meaning of the user’s text aligns with a song’s lyrics, followed
by an assessment of how well the musical qualities fit the user’s emotional
tone. The emotional tone is translated into target audio traits, with brighter
values for a positive mood, softer values for a negative one, and balanced
qualities for a neutral state, and these are compared with each song’s actual
audio features. A smaller portion of the score is based on whether the song’s
overall sentiment matches the user’s mood, giving a full point for a match and
none if it differs.


3. MERGING - All three components are combined into a single score made up
mostly of the text similarity value, with additional influence from the musical
fit and a smaller contribution from the sentiment alignment. This approach
naturally favours songs that reflect the user’s situation in meaning, supports
those that match the intended mood, and gives a slight advantage to those that
share the same emotional tone.


4. RANKING - The songs are then ordered from the highest combined score to the
lowest, and the top selections are returned. This entire calculation is carried
out in the recommend function where the text, audio, and sentiment components
are brought together into one final score.

""")


print("""
Example: User says "I feel low today, work is overwhelming"

1. ANALYSIS:
   Sentiment: NEGATIVE (0.85 confidence)
   Context Embedding: [0.23, -0.45, 0.67, ...]

2. FOR EACH SONG, CALCULATE:

   Song A: "Sad Song" by Artist X

   Hybrid Score = (0.7 × 0.72) + (0.3 × 0.68) + (0.1 × 1.0)
                = 0.504 + 0.204 + 0.1
                = 0.808

   Song B: "Happy Song" by Artist Y

   Hybrid Score = (0.7 × 0.35) + (0.3 × 0.25) + (0.1 × 0.0)
                = 0.245 + 0.075 + 0.0
                = 0.32

   Result: Song A (0.808) ranks higher than Song B (0.32)
""")

print("\nBy merging, we consider both language and feelings")



MERGING PROCESS:

1. USER INPUT ANALYSIS - It first identifies whether the tone is positive,
negative, or neutral, and then interprets the overall context by creating a
detailed numerical representation of the meaning.


2. SIMILARITY CALCULATIONS - The system places most of its emphasis on how
closely the meaning of the user’s text aligns with a song’s lyrics, followed
by an assessment of how well the musical qualities fit the user’s emotional
tone. The emotional tone is translated into target audio traits, with brighter
values for a positive mood, softer values for a negative one, and balanced
qualities for a neutral state, and these are compared with each song’s actual
audio features. A smaller portion of the score is based on whether the song’s
overall sentiment matches the user’s mood, giving a full point for a match and
none if it differs.


3. MERGING - All three components are combined into a single score made up
mostly of the text similarity value, with additional influence f

##9 - Contextual Recommendation System




In [ ]:
class ContextualRecommender:

    def __init__(self, df, sentiment_model, embedding_model):
        self.df = df.copy()
        self.sentiment_model = sentiment_model
        self.embedding_model = embedding_model
        self.audio_features = [f'{f}_scaled' for f in
                              ['valence', 'energy', 'danceability', 'acousticness',
                               'instrumentalness', 'liveness', 'speechiness']
                              if f'{f}_scaled' in df.columns]

        logger.info(f"Recommender has been initialized with {len(df)} songs")

    def analyze_input(self, user_text):
        """First, we anakyse the sentiment and context"""
        clean_text = user_text.lower()
        clean_text = re.sub(r'[^a-z\s]', '', clean_text)
        clean_text = re.sub(r'\s+', ' ', clean_text).strip()

        # SENTIMENT ANALYSIS
        try:
            sentiment_result = self.sentiment_model(user_text[:512])[0]
            sentiment_label = sentiment_result['label'].upper()
            if 'POSITIVE' in sentiment_label or 'POS' in sentiment_label:
                sentiment_label = 'POSITIVE'
            elif 'NEGATIVE' in sentiment_label or 'NEG' in sentiment_label:
                sentiment_label = 'NEGATIVE'
            else:
                sentiment_label = 'NEUTRAL'
            sentiment_score = sentiment_result['score']
        except:
            sentiment_label = 'NEUTRAL'
            sentiment_score = 0.5

        # CONTEXT ANALYSIS
        embedding = self.embedding_model.encode([user_text], show_progress_bar=False)[0]

        return {
            'sentiment_label': sentiment_label,
            'sentiment_score': sentiment_score,
            'embedding': np.array(embedding)
        }

    def recommend(self, user_text, top_n=5, text_weight=0.7, audio_weight=0.3):
        # We now merge the two
        # Step 1 - Get both sentiment and context
        user_analysis = self.analyze_input(user_text)

        # Step 2 - We check for similarity in context
        # Compare our semantic embedding with all song lyrics embeddings
        all_embeddings = np.array(self.df['embedding'].tolist())
        text_sim = cosine_similarity([user_analysis['embedding']], all_embeddings)[0]

        # Step 3 - we check for the similarity in the audio
        if user_analysis['sentiment_label'] == 'POSITIVE':
            # If we determine that the user is positive, we look for upbeat, energetic songs
            target_audio = np.array([0.7, 0.6, 0.6, 0.3, 0.2, 0.3, 0.1])
        elif user_analysis['sentiment_label'] == 'NEGATIVE':
            #  If we determine that the user is negative, then we head towards calmer, lower energy songs
            target_audio = np.array([0.3, 0.4, 0.4, 0.5, 0.3, 0.2, 0.1])
        else:
            # Neutral  features
            target_audio = np.array([0.5, 0.5, 0.5, 0.4, 0.3, 0.3, 0.1])

        # Adjust the target_audio to match available features
        if len(target_audio) > len(self.audio_features):
            target_audio = target_audio[:len(self.audio_features)]
        elif len(target_audio) < len(self.audio_features):
            target_audio = np.pad(target_audio, (0, len(self.audio_features) - len(target_audio)), 'constant')

        # Compare target audio with song audio features based on the sentiments calculated
        all_audio = self.df[self.audio_features].values
        audio_sim = cosine_similarity([target_audio], all_audio)[0]


        # Step 4 - we match for sentiments, i.e. we check if song's sentiment label matches user's sentiment
        sentiment_match = (self.df['sentiment_label'] == user_analysis['sentiment_label']).astype(float)

        # We merge all 3 scores
        hybrid_scores = (
            text_weight * text_sim +
            audio_weight * audio_sim +
            SENTIMENT_WEIGHT * sentiment_match
        )
        # hybrid_scores depict the final combined score for each song


        # Step 5 - We provide the ranking based on the merged score
        top_indices = np.argsort(hybrid_scores)[::-1][:top_n]

        # Create results with all similarity scores
        results = self.df.iloc[top_indices][['name', 'artists', 'sentiment_label',
                                           'valence', 'energy', 'danceability']].copy()
        results = results.reset_index(drop=True)

        results['text_similarity'] = pd.Series(text_sim).iloc[top_indices].to_numpy()
        results['audio_similarity'] = pd.Series(audio_sim).iloc[top_indices].to_numpy()
        results['hybrid_score']     = pd.Series(hybrid_scores).iloc[top_indices].to_numpy()

        match_reasons = []
        for _, row in results.iterrows():
            reasons = []
            if row['sentiment_label'] == user_analysis['sentiment_label']:
                reasons.append("matching sentiment")
            if row['text_similarity'] > 0.3:
                reasons.append("similar theme")
            if row['audio_similarity'] > 0.7:
                reasons.append("matching mood")
            match_reasons.append("; ".join(reasons) if reasons else "general match")
        results['match_reason'] = match_reasons

        return results, user_analysis

print("ContextualRecommender class has been defined with  merging logic")

ContextualRecommender class has been defined with  merging logic


##10 - Practical Example


In [ ]:
# Initialize recommender

# Ensure the recommender is always initialized with the latest class definition
recommender = ContextualRecommender(df_processed, sentiment_model, embedding_model)

example_input = "I feel low today, there's a lot of work and life has become boring"


print(f"\nUser Input: '{example_input}'\n")
# Get recommendations (calling it once to print results and once to store them)
recommendations_print, analysis_print = recommender.recommend(example_input, top_n=3)

print(recommendations_print) # Print the DataFrame result

recommendations, analysis = recommender.recommend(example_input, top_n=3)

print(f"Analysis results:")
print(f"sentiment detected: {analysis['sentiment_label']}")
print(f"confidence: {analysis['sentiment_score']:.2%}")
print(f"Embedding shape: {analysis['embedding'].shape} (768 dimensions)")

print(f"\nTop 3 Recommendations with scores and explanations:\n")

for i, (_, row) in enumerate(recommendations.iterrows(), 1):
    print(f"{i}. {row['name']} - {row['artists']}")
    print(f"    Text Similarity:     {row['text_similarity']:.3f} (Context match - {TEXT_WEIGHT*100:.0f}% weight)")
    print(f"    Audio Similarity:    {row['audio_similarity']:.3f} (Mood match - {AUDIO_WEIGHT*100:.0f}% weight)")
    print(f"    Sentiment Match:     {1.0 if row['sentiment_label'] == analysis['sentiment_label'] else 0.0:.1f} (Direct match - {SENTIMENT_WEIGHT*100:.0f}% weight)")
    print(f"    HYBRID SCORE:        {row['hybrid_score']:.3f} (MERGED SCORE)\n")

    # Show calculation
    text_contrib = TEXT_WEIGHT * row['text_similarity']
    audio_contrib = AUDIO_WEIGHT * row['audio_similarity']
    sentiment_contrib = SENTIMENT_WEIGHT * (1.0 if row['sentiment_label'] == analysis['sentiment_label'] else 0.0)

    print(f"      Calculation: {TEXT_WEIGHT}×{row['text_similarity']:.3f} + {AUDIO_WEIGHT}×{row['audio_similarity']:.3f} + {SENTIMENT_WEIGHT}×{1.0 if row['sentiment_label'] == analysis['sentiment_label'] else 0.0:.1f}")
    print(f"                 = {text_contrib:.3f} + {audio_contrib:.3f} + {sentiment_contrib:.3f}")
    print(f"                 = {row['hybrid_score']:.3f}")


User Input: 'I feel low today, there's a lot of work and life has become boring'

               name           artists sentiment_label  valence  energy  \
0  I'm Always Tired       Joyce Manor        NEGATIVE    0.491   0.846   
1                TV            Elohim        NEGATIVE    0.527   0.503   
2        Tough Luck  nothing,nowhere.        NEGATIVE    0.304   0.598   

   danceability  text_similarity  audio_similarity  hybrid_score  \
0         0.549         0.443672          0.956207      0.697433   
1         0.754         0.440355          0.895571      0.676920   
2         0.557         0.342594          0.933572      0.619887   

                                       match_reason  
0  matching sentiment; similar theme; matching mood  
1  matching sentiment; similar theme; matching mood  
2  matching sentiment; similar theme; matching mood  
Analysis results:
sentiment detected: NEGATIVE
confidence: 91.72%
Embedding shape: (768,) (768 dimensions)

Top 3 Recommendations w

## 11 - Try it yourself!

Change the text below to get recommendations for your situation:


In [ ]:

your_input = "Gym songs"

recommender = ContextualRecommender(df_processed, sentiment_model, embedding_model)

print(f"\nYOu  Provided: '{your_input}'\n")
recommendations_print, analysis_print = recommender.recommend(your_input, top_n=3)

print(recommendations_print) # Print the DataFrame result

recommendations, analysis = recommender.recommend(your_input, top_n=3)

print(f"Analysis results:")
print(f"sentiment detected: {analysis['sentiment_label']}")
print(f"confidence: {analysis['sentiment_score']:.2%}")
print(f"Embedding shape: {analysis['embedding'].shape} (768 dimensions)")

print(f"\nTop 3 Recommendations with scores and explanations:\n")

for i, (_, row) in enumerate(recommendations.iterrows(), 1):
    print(f"{i}. {row['name']} - {row['artists']}")
    print(f"    Text Similarity:     {row['text_similarity']:.3f} (Context match - {TEXT_WEIGHT*100:.0f}% weight)")
    print(f"    Audio Similarity:    {row['audio_similarity']:.3f} (Mood match - {AUDIO_WEIGHT*100:.0f}% weight)")
    print(f"    Sentiment Match:     {1.0 if row['sentiment_label'] == analysis['sentiment_label'] else 0.0:.1f} (Direct match - {SENTIMENT_WEIGHT*100:.0f}% weight)")
    print(f"    HYBRID SCORE:        {row['hybrid_score']:.3f} (MERGED SCORE)\n")

    # Show calculation
    text_contrib = TEXT_WEIGHT * row['text_similarity']
    audio_contrib = AUDIO_WEIGHT * row['audio_similarity']
    sentiment_contrib = SENTIMENT_WEIGHT * (1.0 if row['sentiment_label'] == analysis['sentiment_label'] else 0.0)

    print(f"      Calculation: {TEXT_WEIGHT}×{row['text_similarity']:.3f} + {AUDIO_WEIGHT}×{row['audio_similarity']:.3f} + {SENTIMENT_WEIGHT}×{1.0 if row['sentiment_label'] == analysis['sentiment_label'] else 0.0:.1f}")
    print(f"                 = {text_contrib:.3f} + {audio_contrib:.3f} + {sentiment_contrib:.3f}")
    print(f"                 = {row['hybrid_score']:.3f}")


YOu  Provided: 'Gym songs'

           name             artists sentiment_label  valence  energy  \
0  Baby Workout   ['George Benson']         NEUTRAL    0.711   0.454   
1       Pushin'  ['Frank Stallone']         NEUTRAL    0.819   0.714   
2  Strike It Up           Black Box         NEUTRAL    0.915   0.862   

   danceability  text_similarity  audio_similarity  hybrid_score  \
0         0.760         0.623399          0.865962      0.796168   
1         0.795         0.388576          0.914403      0.646324   
2         0.876         0.414143          0.839770      0.641831   

                                       match_reason  
0  matching sentiment; similar theme; matching mood  
1  matching sentiment; similar theme; matching mood  
2  matching sentiment; similar theme; matching mood  
Analysis results:
sentiment detected: NEUTRAL
confidence: 73.10%
Embedding shape: (768,) (768 dimensions)

Top 3 Recommendations with scores and explanations:

1. Baby Workout - ['George Benson